In [25]:
from config import fund_dict
import requests
import pandas as pd
from helper import create_holdings_df, process_scraped_data
import traceback
import numpy as np
import seaborn as sns
import traceback
from datetime import datetime, timedelta

figi_api_key = '34dada0c-5a22-4f5a-a095-739afe8fe591' 

In [31]:
ninty_days_ago = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')
#ninty_days_ago = (datetime.strptime("2023-03-30", '%Y-%m-%d') - timedelta(days=90)).strftime('%Y-%m-%d')

ninty_days_ago

'2022-12-30'

In [38]:
def process_scraped_data_(df_all_):
    df_all_['nameOfIssuer_link'] =df_all_['nameOfIssuer'].apply(
    lambda x: str('''<a href="http://www.google.com/search?q=stock price {}" 
        target="_blank">{}</a>'''.format(x, x)))
    periods = list(set(df_all_.reportDate.values))
    ninty_days_ago_ = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')
    for period in periods:
        if period > ninty_days_ago_:
            periods.remove(period)
    periods.sort(reverse=True)
    print('periods: ',periods)
    df_all_.reset_index().tail()
    #df_all_[df_all_.reportDate == periods[0]].reset_index().tail()
    df_current = df_all_[df_all_.reportDate == periods[0]].reset_index(drop=True)
    print('df_current.tail(): ',df_current.tail())
    df_previous = df_all_[df_all_.reportDate == periods[1]].reset_index(drop=True)
    df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
    df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
    df_diff = df_current_g - df_previous_g
    df_heavy_ = df_current_g
    df_hot_ = df_diff[df_diff.value > 0]
    df_cold_ = df_diff[df_diff.value < 0]
    
    return df_heavy_, df_hot_, df_cold_

In [11]:
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df(cik_=cik,fund_name_=fund_name)
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print(fund_name, cik)
    except:
        print(traceback.format_exc())
        continue

#df_all.to_csv('data_by_stocks.csv',index=False)


https://data.sec.gov/submissions/CIK0001358706.json
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-23-000651.txt
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-22-002366.txt
ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
https://data.sec.gov/submissions/CIK0001112520.json
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-22-000015.txt
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-22-000013.txt
AKRE CAPITAL MANAGEMENT 1112520
https://data.sec.gov/submissions/CIK0001491072.json
https://www.sec.gov/Archives/edgar/data/1491072/0001491072-22-000007.txt
Traceback (most recent call last):
  File "/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_14717/1766246574.py", line 6, in <module>
    df_fund = create_holdings_df(cik_=cik,fund_name_=fund_name)
  File "/Users/willsolo/Documents/GitHub/stockFries_gha/notebooks/helper.py", line 48, in create_holdings_df
    df_new = pd.DataFrame(xml_doc_['informationTable']['infoTable'])
KeyError: 'informatio

In [39]:
df_heavy, df_hot, df_cold = process_scraped_data_(df_all)

periods:  ['2022-09-30', '2022-06-30', '2020-06-30', '2020-03-31', '2019-12-31', '2018-12-31', '2018-09-30']
df_current.tail():                        nameOfIssuer      titleOfClass      cusip  value  \
4686  TWELVE SEAS INVESTMENT CO II          COM CL A  90118T106   5637   
4687                    VELO3D INC  *W EXP 09/29/202  92259N112     81   
4688                      VIEW INC  *W EXP 03/08/202  92671V114      8   
4689                    YANDEX N V       SHS CLASS A  N97284108   1313   
4690          YPF SOCIEDAD ANONIMA     SPON ADR CL D  984245100   5601   

                                        shrsOrPrnAmt investmentDiscretion  \
4686  {'sshPrnamt': '572545', 'sshPrnamtType': 'SH'}                 SOLE   
4687  {'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}                 SOLE   
4688  {'sshPrnamt': '105000', 'sshPrnamtType': 'SH'}                 SOLE   
4689  {'sshPrnamt': '429023', 'sshPrnamtType': 'SH'}                 SOLE   
4690  {'sshPrnamt': '901888', 'sshPrnamtT

/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_14717/3660602311.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_14717/3660602311.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)


In [40]:
df_cold.tail()

,,,value,sshPrnamt
nameOfIssuer,nameOfIssuer_link,cusip,,
ZSCALER INC,"<a href=""http://www.google.com/search?q=stock price ZSCALER INC"" \n target=""_blank"">ZSCALER INC</a>",98980G102,-4798585.0,-42124.0
ZTO EXPRESS CAYMAN INC,"<a href=""http://www.google.com/search?q=stock price ZTO EXPRESS CAYMAN INC"" \n target=""_blank"">ZTO EXPRESS CAYMAN INC</a>",98980A105,-5538190.0,-185660.0
"aTyr Pharma, Inc.","<a href=""http://www.google.com/search?q=stock price aTyr Pharma, Inc."" \n target=""_blank"">aTyr Pharma, Inc.</a>",002120202,-456.0,-173034.0
argenx SE,"<a href=""http://www.google.com/search?q=stock price argenx SE"" \n target=""_blank"">argenx SE</a>",04016X101,-325268.0,-839398.0
"iTeos Therapeutics, Inc.","<a href=""http://www.google.com/search?q=stock price iTeos Therapeutics, Inc."" \n target=""_blank"">iTeos Therapeutics, Inc.</a>",46565G104,-233.0,0.0


In [23]:
print(df_all.columns)
df_all[df_all['reportDate']=='2022-12-31'].reset_index().tail()

df_all[df_all['reportDate']=='2022-09-30'].reset_index().tail()

Index(['nameOfIssuer', 'titleOfClass', 'cusip', 'value', 'shrsOrPrnAmt',
       'investmentDiscretion', 'votingAuthority', 'sshPrnamt', 'sshPrnamtType',
       'cik', 'filingDate', 'reportDate', 'fund_name', 'putCall',
       'otherManager', 'nameOfIssuer_link'],
      dtype='object')


,index,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,putCall,otherManager,nameOfIssuer_link
4686,9644,TWELVE SEAS INVESTMENT CO II,COM CL A,90118T106,5637,"{'sshPrnamt': '572545', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '572545', 'Shared': '0', 'None': '0'}",572545,SH,1348145,2022-11-14,2022-09-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
4687,9645,VELO3D INC,*W EXP 09/29/202,92259N112,81,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2022-11-14,2022-09-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
4688,9646,VIEW INC,*W EXP 03/08/202,92671V114,8,"{'sshPrnamt': '105000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '105000', 'Shared': '0', 'None': '0'}",105000,SH,1348145,2022-11-14,2022-09-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
4689,9647,YANDEX N V,SHS CLASS A,N97284108,1313,"{'sshPrnamt': '429023', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '429023', 'Shared': '0', 'None': '0'}",429023,SH,1348145,2022-11-14,2022-09-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
4690,9648,YPF SOCIEDAD ANONIMA,SPON ADR CL D,984245100,5601,"{'sshPrnamt': '901888', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '901888', 'Shared': '0', 'None': '0'}",901888,SH,1348145,2022-11-14,2022-09-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."


In [17]:
df_heavy.tail()

,,,value,sshPrnamt
nameOfIssuer,nameOfIssuer_link,cusip,,
CAMPING WORLD HLDGS INC,"<a href=""http://www.google.com/search?q=stock price CAMPING WORLD HLDGS INC"" \n target=""_blank"">CAMPING WORLD HLDGS INC</a>",13462K109,114028595,5108808
U HAUL HOLDING COMPANY,"<a href=""http://www.google.com/search?q=stock price U HAUL HOLDING COMPANY"" \n target=""_blank"">U HAUL HOLDING COMPANY</a>",023586100,24498955,407027
CANTALOUPE INC,"<a href=""http://www.google.com/search?q=stock price CANTALOUPE INC"" \n target=""_blank"">CANTALOUPE INC</a>",138103106,13441500,3090000
USCB FINANCIAL HOLDINGS INC,"<a href=""http://www.google.com/search?q=stock price USCB FINANCIAL HOLDINGS INC"" \n target=""_blank"">USCB FINANCIAL HOLDINGS INC</a>",90355N101,7918837,649085
NUVATION BIO INC,"<a href=""http://www.google.com/search?q=stock price NUVATION BIO INC"" \n target=""_blank"">NUVATION BIO INC</a>",67080N101,7318105,3811513


In [12]:
df_all.tail()

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,putCall,otherManager,nameOfIssuer_link
9710,TWELVE SEAS INVESTMENT CO II,*W EXP 03/02/202,90118T114,16,"{'sshPrnamt': '190848', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '190848', 'Shared': '0', 'None': '0'}",190848,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9711,VELO3D INC,*W EXP 12/01/202,92259N112,34,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9712,VIEW INC,*W EXP 03/08/202,92671V114,16,"{'sshPrnamt': '105000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '105000', 'Shared': '0', 'None': '0'}",105000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9713,WARRIOR TECHNOLOGIES ACQUI C,*W EXP 03/31/202,936273119,8,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9714,YANDEX N V,SHS CLASS A,N97284108,1313,"{'sshPrnamt': '429023', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '429023', 'Shared': '0', 'None': '0'}",429023,SH,1348145,2022-08-15,2022-06-30,VR ADVISORY SERVICES,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."


In [9]:
df_all[df_all['putCall'].notnull()].reset_index().tail()

,index,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,putCall,otherManager,nameOfIssuer_link
1792,9077,ISHARES TR,RUSSELL 2000 ETF,464287955,37,"{'sshPrnamt': '920000', 'sshPrnamtType': 'SH'}",DFND,"{'Sole': '920000', 'Shared': '0', 'None': '0'}",920000,SH,1517137,2022-08-12,2022-06-30,STARBOARD VALUE LP,Put,"1, 2, 3, 4, 5","<a href=""http://www.google.com/search?q=stock ..."
1793,9078,ISHARES TR,RUSSELL 2000 ETF,464287955,5451,"{'sshPrnamt': '1150000', 'sshPrnamtType': 'SH'}",DFND,"{'Sole': '1150000', 'Shared': '0', 'None': '0'}",1150000,SH,1517137,2022-08-12,2022-06-30,STARBOARD VALUE LP,Put,"1, 2, 3, 4, 5","<a href=""http://www.google.com/search?q=stock ..."
1794,9079,ISHARES TR,RUSSELL 2000 ETF,464287955,1026,"{'sshPrnamt': '200000', 'sshPrnamtType': 'SH'}",DFND,"{'Sole': '200000', 'Shared': '0', 'None': '0'}",200000,SH,1517137,2022-08-12,2022-06-30,STARBOARD VALUE LP,Put,"1, 2, 3, 4, 5","<a href=""http://www.google.com/search?q=stock ..."
1795,9168,ISHARES RUSSELL 2000 ETF,SHS,464287655,4295,"{'sshPrnamt': '30000', 'sshPrnamtType': 'SH'}",DFND,"{'Sole': '0', 'Shared': '0', 'None': '0'}",30000,SH,0001030815,2020-08-14,2020-06-30,SYMPHONY ASSET MANAGEMENT LLC,Put,NaN,"<a href=""http://www.google.com/search?q=stock ..."
1796,9184,SPDR S&P 500 ETF TRUST,SHS,78462F103,10793,"{'sshPrnamt': '35000', 'sshPrnamtType': 'SH'}",DFND,"{'Sole': '0', 'Shared': '0', 'None': '0'}",35000,SH,0001030815,2020-08-14,2020-06-30,SYMPHONY ASSET MANAGEMENT LLC,Put,NaN,"<a href=""http://www.google.com/search?q=stock ..."


In [3]:
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df()
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print(fund_name, cik)
    except:
        print(traceback.format_exc())
        continue
df_all['nameOfIssuer_link'] =df_all['nameOfIssuer'].apply(
    lambda x: str('''<a href="http://www.google.com/search?q=stock price {}">{}</a>''' \
        .format(x, x)))
    

df_heavy, df_hot, df_cold = process_scraped_data(df_all)

https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
AKRE CAPITAL MANAGEMENT 1112520
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
ANSON FUNDS 1491072
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-002482.txt
https://www.sec.gov/Archives/edgar/data/1535392/0001172661-22-001938.txt
APPALOOSA LP 1656456
https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/153

/Users/willsolo/Documents/GitHub/stockFries_gha/notebooks/helper.py:97: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
/Users/willsolo/Documents/GitHub/stockFries_gha/notebooks/helper.py:98: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)


In [5]:
df_all.tail()
#set(df_all.sshPrnamtType.tolist())

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,nameOfIssuer_link
50003,WINVEST ACQUISITION CORP,*W EXP 08/09/202,97655B117,4843,"{'sshPrnamt': '974500', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '974500', 'Shared': '0', 'None': '0'}",974500,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50004,XPAC ACQUISITION CORP,CLASS A ORD,G9831X106,4880,"{'sshPrnamt': '500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '500000', 'Shared': '0', 'None': '0'}",500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50005,YOTTA ACQUISITION CORPORATIO,UNIT 03/15/2027,98741Y202,4487,"{'sshPrnamt': '450000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '450000', 'Shared': '0', 'None': '0'}",450000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50006,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,120,"{'sshPrnamt': '1500000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '1500000', 'Shared': '0', 'None': '0'}",1500000,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."
50007,ZENDESK INC,COM,98936J101,11701,"{'sshPrnamt': '157974', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '157974', 'Shared': '0', 'None': '0'}",157974,SH,1535392,2022-08-15,2022-06-30,Mangrove Partners,"<a href=""http://www.google.com/search?q=stock ..."


In [ ]:
df_recent_options = pd.read_json(hedge_fund_json_dg, orient='split')
df_puts_only = df_recent_options.drop_duplicates(subset=['CUSIP','HF_Name'], keep=False)
df_puts_only = df_puts_only[(df_puts_only['Option_Type']=='Put') & (df_puts_only['HF_Name']!='CAPITAL FUND MANAGEMENT')]
print(len(set(df_puts_only.HF_Name.tolist())))
df_puts_only.sort_values('Company').reset_index().tail(100)
df_puts_only.groupby(['CUSIP','Company','Option_Type'],as_index=False).agg(
        {'Shares': 'sum', 'Value': 'sum', 'HF_Name': 'unique'}).sort_values('Value').tail(20)